In [1]:
import os

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
import torch.optim as optim

from collections import Counter
from datetime import datetime
from torch import nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing, DataParallel
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

from DataClasses import lmdb_dataset, Dataset, choose_dataloader
from ModelFunctions import my_reshape, multigpu_available, predict, train, evaluate, inference

import tqdm

In [2]:
import sys
sys.path.append(os.path.expanduser('../ocpmodels/models'))
sys.path.append(os.path.expanduser('../../ocp_airi'))

#from spinconv import spinconv
from spinconv_with_embeds_single import spinconv

In [3]:
#вызывается каждый раз, когда датасет отдаёт элемент (систему)
#делаем из данных матрицу векторов-атомов, список рёбер (edge_index) и матрицу векторов-рёбер; надо писать свою функцию для каждой сети
def preprocessing(system):
    keys = ['pos', 'atomic_numbers', 'cell', 'natoms', 'sid', 'y_relaxed']
    features_dict = {}
    for key in keys:
        features_dict[key] = system[key]
    return Data(**features_dict)

In [4]:
m_table = pd.read_csv('https://gist.githubusercontent.com/GoodmanSciences/c2dd862cd38f21b0ad36b8f96b4bf1ee/raw/1d92663004489a5b6926e944c1b3d9ec5c40900e/Periodic%2520Table%2520of%2520Elements.csv')

In [5]:
#config
subsample = 'ood_both'
batch_size = 63
num_workers = 0

features_cols = ['feature_1']

target_col = 'y_relaxed'
lr = 0.001
epochs = 30

In [6]:
# #чтобы тензор по умолчанию заводился на куде
# if torch.cuda.is_available():
#     torch.set_default_tensor_type('torch.cuda.FloatTensor')
#     print('cuda')

In [7]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

cuda


In [8]:
#инициализируем валидационный датасет и валидационный итератор
val_dataset_file_path = os.path.expanduser(f"../../ocp_datasets/data/is2re/all/val_{subsample}/data.lmdb")

valid_set = Dataset(val_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)
valid_generator = choose_dataloader(valid_set, batch_size=batch_size, num_workers=num_workers)

In [9]:
try:
    lmdb_dataset(train_dataset_file_path).describe()
except:
    pass

In [14]:
#model
# model = spinconv(None, None, 1, otf_graph=True, regress_forces=False)
model = spinconv(None, None, 1, otf_graph=True, regress_forces=False, custom_embedding_value=[33])
# checkpoint = '2021-09-25-11-23-25_epoch_15_.pickle'
# model_type = 'id'
checkpoint = '2021-10-05-14-31-19_epoch_13_.pickle'
model_type = 'electronegativity'
folder = '../checkpoints_for_analysis/'
model_chp = torch.load(folder + checkpoint,  map_location=device)
state_dict = model_chp.module.state_dict()
# state_dict = model_chp.state_dict()
# new_state_dict = OrderedDict()
# for k, v in state_dict.items():
#     name = k[7:] # remove `module.`
#     new_state_dict[name] = v
if multigpu_available():
    model = DataParallel(model)

#optimizer and loss
optimizer = optim.AdamW(model.parameters(), lr=lr)
criterion = nn.L1Loss()

#переносим на куду если она есть
model = model.to(device)
criterion = criterion.to(device)

# model.load_state_dict(new_state_dict)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [15]:
iterator = valid_generator

In [16]:
result = torch.tensor([]).to(device)

multigpu_mode = multigpu_available()

model.eval()

with torch.no_grad():
    for batch in tqdm.tqdm(iterator):
        predictions, sids = predict(model, batch, multigpu_mode, device, inference=True)
        sids = my_reshape(sids)
        predictions = my_reshape(predictions)
        y_true = my_reshape(batch['y_relaxed'])
        mini_submit = torch.cat((sids, predictions, y_true), dim=1)
        result = torch.cat((result, mini_submit))        

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 397/397 [02:34<00:00,  2.57it/s]


In [17]:
df = pd.DataFrame(result, columns=['sid', 'pred', 'target'])

In [18]:
for column in ['sid', 'pred', 'target']:
    df[column] = df[column].apply(lambda x: x.item())
    
df['sid'] = df['sid'].apply(int)
df['index'] = df.index

In [19]:
dataset = lmdb_dataset(val_dataset_file_path)

In [20]:
def get_composition(row):
    i = int(row['index'])
    from_lmdb = dataset[i]
    assert(int(from_lmdb['sid']) == int(row['sid']))
    atomic_numbers = from_lmdb['atomic_numbers']
    counter = Counter(atomic_numbers.long().tolist())
    composition = []
    for key in counter:
        element = m_table.iloc[key-1]['Symbol']
        composition.append(element)
        composition.append(str(counter[key]))
    return '_'.join(composition)

In [21]:
%%time
df['composition'] = df.apply(get_composition, axis=1)

CPU times: user 27.6 s, sys: 244 ms, total: 27.9 s
Wall time: 27.8 s


In [22]:
df

,sid,pred,target,index,composition
0,2574092,-0.937665,-1.500569,0,Ni_72_Sb_12_Te_24_C_1_H_1_O_1
1,2522358,-2.405628,-2.846116,1,Hf_40_Re_20_Ru_20_C_1_H_1_O_1
2,2543745,-8.981351,-1.470944,2,H_32_N_50_P_24_C_2
3,2573082,-3.981120,-5.303689,3,Nb_24_Co_36_Ge_12_C_2_H_3_O_2
4,2540810,-0.101637,1.946157,4,Nb_48_Se_72_N_1_H_2
...,...,...,...,...,...
24982,2009063,0.018295,-0.577668,24982,Pd_16_Au_48_C_1_H_1_O_1
24983,2541962,0.082157,0.087245,24983,Al_16_Co_32_Cr_16_C_1_H_1_O_1
24984,2530077,-1.922614,-1.052784,24984,Co_36_Y_36_C_1_H_1_O_1
24985,1445329,-5.048871,-4.045752,24985,Fe_72_N_26_C_2_H_8


In [25]:
df.to_csv(f'../results_csv/{model_type}_val_{subsample}.csv')